<a href="https://colab.research.google.com/github/Manav-Mistry/Object_Detection_and_Localization/blob/master/Faster_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Object Detection and Localization using R-CNN

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

### downloading module from tensorflow hub

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1";

detector = hub.load(module_handle).signatures['default']

### function for download and resize

In [ ]:
def download_and_resize_image(url, new_width=256, new_height=256):
  """
  Fetches an image online, resizes it and saves it locally.

  Args:
    url (string) -- link to the image
    new_width (int) -- size iin pixels used for resizing the width of the image
    new_height (int) -- size in pixels used for resizing the length of the image

  Returns:
    (string) -- path to saved image
  """

  # create a temporary file ending with ".jpg"
  _, filename = tempfile.mkstemp(suffix=".jpg")

  # opens the given URL
  response = urlopen(url)

  # reads the image fetched from the URL
  image_data = response.read()

  # puts the image data in memory buffer
  image_data = BytesIO(image_data)

  # opens the image
  pil_image = Image.open(image_data)

  # resizes the image. will crop if aspect ratio is different.
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)

  # converts to the RGB colorspace
  pil_image_rgb = pil_image.convert("RGB")

  # saves the image to the temporary file created earlier
  pil_image_rgb.save(filename, format="JPEG", quality=90)

  print("Image downloaded to %s." % filename)

  return filename

In [ ]:
img_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG";

downloaded_image_path = download_and_resize_image(
    url = img_url,
    new_width = 3872,
    new_height = 2592
)

<ipython-input-3-081da85fe049>:30: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)


Image downloaded to /tmp/tmpgys_gqiy.jpg.


### Load_img to convert tensor to image
### run_detector to detect object and draw bounding lines


In [ ]:
def load_img(path):
  """
  Loads a JPEG image and converts it to a tensor.

  Args:
    path (string) -- path to a locally saved JPEG image

  Returns:
    (tensor) -- an image tensor
  """

  # read the file
  img = tf.io.read_file(path)

  # convert to a tensor
  img = tf.image.decode_jpeg(img, channels=3)

  return img

def run_detector(detector, path):
  """
  Runs inference on a local files using an object detection model.

  Args:
    detector (model) -- an object detection model loaded from TF hub
    path (string) -- path to an image saved locally
  """

  # load an image tensor from a local file path
  img = load_img(path)

  # add a batch dimension in front of the tensor
  converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  # run inference using the model
  result = detector(converted_img)

  # save the results in a dictionary
  result = {key: value.numpy() for key, value in result.items()}

  # print results
  print("Found %d objects." % len(result["detection_scores"]))

  print(result["detection_scores"])
  print(result["detection_class_entities"])
  print(result["detection_boxes"])

In [ ]:
run_detector(detector, downloaded_image_path)

### Clone the tensorflow models repository

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models


Cloning into 'models'...
remote: Enumerating objects: 3984, done.
remote: Counting objects: 100% (3984/3984), done.
remote: Compressing objects: 100% (3091/3091), done.
remote: Total 3984 (delta 1155), reused 1941 (delta 836), pack-reused 0
Receiving objects: 100% (3984/3984), 49.75 MiB | 6.62 MiB/s, done.
Resolving deltas: 100% (1155/1155), done.


In [ ]:
import os
print(os.getcwd())

new_directory_path = '/content/models/research'  # Change this to the desired directory

# Change the working directory
os.chdir(new_directory_path)
print(os.getcwd())

In [ ]:
# compile the files to get .py file from .proto

!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 59.5 MB/s eta 0:00:0

In [ ]:
!pwd

/content/models/research


In [ ]:
protos_path = 'object_detection/protos'  # Change this to the desired directory

# Change the working directory
os.chdir(protos_path)
print(os.getcwd())

/content/models/research/object_detection/protos


In [ ]:
!protoc string_int_label_map.proto --python_out=.

In [27]:
from object_detection.utils import label_map_utils
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

ImportError: ignored

## Models upload to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print()